In [117]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import pickle
import os 

In [127]:
all_data = pd.read_csv('.\\ipl_csv2\\all_matches.csv')
all_data['season'].unique()
# all_data = all_data[all_data['ball'] < 6.1]
# all_data = all_data[all_data['innings'] <= 2]

# all_data['total_runs'] = all_data['runs_off_bat'] + all_data['extras']

C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


array(['2007/08', '2009', '2009/10', '2011', '2012', 2012, 2013, 2014,
       2015, 2016, 2017, 2019, 2018, '2018', '2019', '2020/21', '2021'],
      dtype=object)

In [39]:
# all_data.groupby('match_id')['total_runs'].sum()

In [92]:
dataPaths = list(paths.list_files('.\ipl_csv2'))
dataPaths.pop(len(dataPaths) - 2)
dataPaths.pop(len(dataPaths) - 1)

def retrieve_past_data(team_a, team_b, num):
    req_df = []
    stadium_avg = {}
    unique_bowlers = set()
    unique_batsmen = set()

    for match_file in dataPaths:
        try:
            df = pd.read_csv(match_file, index_col='match_id')
            temp_a, temp_b = get_unique_players(df)
            unique_bowlers |= (temp_a) 
            unique_batsmen |= (temp_b) 
            df = preprocess(df)
            stadium_avg = get_stadium_avg(stadium_avg, df)
            if (df.iloc[1]['batting_team'] == team_a or df.iloc[1]['batting_team'] == team_b) and (df.iloc[1]['bowling_team'] == team_a or df.iloc[1]['bowling_team'] == team_b):
                if len(req_df) < num or num == -1:
                    req_df.append(df)
        
        except Exception as e:
            print(e)

    return req_df, stadium_avg, unique_batsmen, unique_bowlers


def preprocess(df):
    pp_df = df[df['ball'] < 6.1]
    pp_df = pp_df[pp_df['innings'] <= 2]

    pp_df['total_runs'] = pp_df['runs_off_bat'] + pp_df['extras']

    return pp_df

#for stadium avg
def get_stadium_avg(stad_dict, df):
    venue = df.iloc[1]['venue']
    runs1 = df.loc[(df['ball'] < 6.1) & (df['innings'] == 1)]['total_runs'].sum()
    runs2 = df.loc[(df['ball'] < 6.1) & (df['innings'] == 2)]['total_runs'].sum()

    
    stad_dict.setdefault(venue,[]).append(runs1)
    stad_dict.setdefault(venue,[]).append(runs2)

    return stad_dict

def get_unique_players(df):
    bowlers = set(df['bowler'].unique().tolist())
    batsmen_striker = set(df['striker'].unique().tolist())
    batsmen_non_striker = set(df['non_striker'].unique().tolist())
    
    batsmen = batsmen_striker | (batsmen_non_striker)

    return bowlers, batsmen


In [126]:
#returns a list of dataframes between the two given teams
req_df, stad_avg, u_batsmen, u_bowler = retrieve_past_data("Chennai Super Kings", -1, -1)
print(len(req_df))

0


In [105]:
print(len(u_batsmen))
print(len(u_bowler))
print(len(u_batsmen | u_bowler))
print(len(req_df))

544
422
583
0


In [128]:
#format the data for visualization (leave it out for now)

stad_avg['M.Chinnaswamy Stadium'].extend(stad_avg['M Chinnaswamy Stadium'])
# stad_avg.pop('M Chinnaswamy Stadium')

stad_avg['MA Chidambaram Stadium'].extend(stad_avg['MA Chidambaram Stadium, Chepauk'])
stad_avg['MA Chidambaram Stadium'].extend(stad_avg['MA Chidambaram Stadium, Chepauk, Chennai'])
# stad_avg.pop('MA Chidambaram Stadium, Chepauk')
# stad_avg.pop('MA Chidambaram Stadium, Chepauk, Chennai')

stad_avg['Punjab Cricket Association IS Bindra Stadium'].extend(stad_avg['Punjab Cricket Association IS Bindra Stadium, Mohali'])
stad_avg['Punjab Cricket Association IS Bindra Stadium'].extend(stad_avg['Punjab Cricket Association Stadium, Mohali'])
# stad_avg.pop('Punjab Cricket Association IS Bindra Stadium, Mohali')
# stad_avg.pop('Punjab Cricket Association Stadium, Mohali')

stad_avg['Rajiv Gandhi International Stadium'].extend(stad_avg['Rajiv Gandhi International Stadium, Uppal'])
# stad_avg.pop('Rajiv Gandhi International Stadium, Uppal')

stad_avg['Arun Jaitley Stadium'].extend(stad_avg['Feroz Shah Kotla'])
# stad_avg.pop('Feroz Shah Kotla')

stad_avg['Wankhede Stadium'].extend(stad_avg['Wankhede Stadium, Mumbai'])
# stad_avg.pop('Wankhede Stadium, Mumbai')

In [129]:
stad_rating = {}
for stad in stad_avg:
    stad_rating[stad] = float(np.mean(stad_avg[stad]))

stad_rating['M Chinnaswamy Stadium'] = stad_rating['M.Chinnaswamy Stadium']
stad_rating['MA Chidambaram Stadium, Chepauk'] = stad_rating['MA Chidambaram Stadium']
stad_rating['MA Chidambaram Stadium, Chepauk, Chennai'] = stad_rating['MA Chidambaram Stadium']
stad_rating['Punjab Cricket Association IS Bindra Stadium, Mohali'] = stad_rating['Punjab Cricket Association IS Bindra Stadium']
stad_rating['Punjab Cricket Association Stadium, Mohali'] = stad_rating['Punjab Cricket Association IS Bindra Stadium']
stad_rating['Rajiv Gandhi International Stadium, Uppal'] = stad_rating['Rajiv Gandhi International Stadium']
stad_rating['Feroz Shah Kotla'] = stad_rating['Arun Jaitley Stadium']
stad_rating['Wankhede Stadium, Mumbai'] = stad_rating['Wankhede Stadium']

print(stad_rating)

{'Rajiv Gandhi International Stadium, Uppal': 45.5859375, 'Maharashtra Cricket Association Stadium': 47.45238095238095, 'Saurashtra Cricket Association Stadium': 55.3, 'Holkar Cricket Stadium': 51.72222222222222, 'M.Chinnaswamy Stadium': 45.55, 'Wankhede Stadium': 45.6554054054054, 'Eden Gardens': 46.26623376623377, 'M Chinnaswamy Stadium': 45.55, 'Feroz Shah Kotla': 47.4527027027027, 'Punjab Cricket Association IS Bindra Stadium, Mohali': 48.42857142857143, 'Green Park': 55.0, 'Punjab Cricket Association IS Bindra Stadium': 48.42857142857143, 'Rajiv Gandhi International Stadium': 45.5859375, 'MA Chidambaram Stadium': 46.40677966101695, 'Sawai Mansingh Stadium': 45.04255319148936, 'Arun Jaitley Stadium': 47.4527027027027, 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 40.96153846153846, 'Sheikh Zayed Stadium': 43.672413793103445, 'Dubai International Cricket Stadium': 44.71212121212121, 'Sharjah Cricket Stadium': 47.02777777777778, 'MA Chidambaram Stadium, Chepauk, Chennai': 46

In [130]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

save_obj(stad_rating, 'stad_record')
test = load_obj('stad_record')
print(test)

{'Rajiv Gandhi International Stadium, Uppal': 45.5859375, 'Maharashtra Cricket Association Stadium': 47.45238095238095, 'Saurashtra Cricket Association Stadium': 55.3, 'Holkar Cricket Stadium': 51.72222222222222, 'M.Chinnaswamy Stadium': 45.55, 'Wankhede Stadium': 45.6554054054054, 'Eden Gardens': 46.26623376623377, 'M Chinnaswamy Stadium': 45.55, 'Feroz Shah Kotla': 47.4527027027027, 'Punjab Cricket Association IS Bindra Stadium, Mohali': 48.42857142857143, 'Green Park': 55.0, 'Punjab Cricket Association IS Bindra Stadium': 48.42857142857143, 'Rajiv Gandhi International Stadium': 45.5859375, 'MA Chidambaram Stadium': 46.40677966101695, 'Sawai Mansingh Stadium': 45.04255319148936, 'Arun Jaitley Stadium': 47.4527027027027, 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 40.96153846153846, 'Sheikh Zayed Stadium': 43.672413793103445, 'Dubai International Cricket Stadium': 44.71212121212121, 'Sharjah Cricket Stadium': 47.02777777777778, 'MA Chidambaram Stadium, Chepauk, Chennai': 46

In [112]:
batsmen_record = {}
bowlers_record = {}

for match_file in dataPaths:
    df_all = pd.read_csv(match_file, index_col='match_id')
    df = df_all.loc[df_all['ball'] < 6.1]

    m_bowl, m_bat = get_unique_players(df) 

    for bat in m_bat:
        select = df.loc[df['striker'].str.contains(str(bat))]
        select_all = df_all.loc[df_all['striker'].str.contains(str(bat))]
        runs_scored = select['runs_off_bat'].sum() + select['extras'].sum()
        runs_scored_after_pp = select_all['runs_off_bat'].sum() + select_all['extras'].sum() - runs_scored
        balls_faced = len(select.index)
        boundaries = len(select.loc[(select['runs_off_bat'] == 6) | (select['runs_off_bat'] == 4)])

        batsmen_record.setdefault(bat,[[],[],[],[]])[0].append(runs_scored)
        batsmen_record.setdefault(bat,[[],[],[],[]])[1].append(balls_faced)
        batsmen_record.setdefault(bat,[[],[],[],[]])[2].append(runs_scored_after_pp)
        batsmen_record.setdefault(bat,[[],[],[],[]])[3].append(boundaries)

    for bowl in m_bowl:
        select = df.loc[df['bowler'].str.contains(str(bowl))]
        total_runs_given = select['runs_off_bat'].sum() + select['extras'].sum()
        extras_given = select['extras'].sum()
        balls_bowled = len(select.index)
        wickets_taken = len(select.loc[select['wicket_type'].notnull()])

        bowlers_record.setdefault(bowl,[[],[],[],[]])[0].append(total_runs_given)
        bowlers_record.setdefault(bowl,[[],[],[],[]])[1].append(extras_given)
        bowlers_record.setdefault(bowl,[[],[],[],[]])[2].append(balls_bowled)
        bowlers_record.setdefault(bowl,[[],[],[],[]])[3].append(wickets_taken)


C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [114]:
# print(bowlers_record)
print(batsmen_record['V Kohli'])

[[27, 28, 28, 0, 12, 27, 21, 6, 5, 1, 10, 21, 37, 27, 14, 18, 9, 28, 37, 35, 12, 4, 7, 21, 3, 25, 29, 12, 34, 9, 19, 9, 13, 23, 25, 16, 7, 6, 1, 6, 13, 11, 12, 17, 4, 5, 6, 21, 5, 15, 26, 4, 7, 10, 0, 23, 1, 2, 23, 14, 4, 10, 42, 1, 6, 10, 13, 22, 3, 19, 3, 6, 2, 12, 4, 16, 18, 10, 3, 22, 6, 24, 25, 12, 15, 11, 0, 4, 8, 5, 15, 5, 40, 11, 0, 9, 4, 1, 1, 0, 4, 1, 1, 5, 9, 0, 13, 22, 0, 5, 12, 6, 1, 27, 12, 15, 14, 19, 44, 1, 8, 12, 18, 24, 18, 29, 30, 14, 12, 21, 16, 8, 18, 20, 26, 17, 0, 11], [19, 19, 25, 1, 14, 17, 15, 6, 9, 3, 13, 16, 20, 18, 12, 15, 10, 16, 18, 22, 11, 9, 12, 10, 9, 24, 16, 11, 20, 10, 17, 8, 8, 15, 7, 7, 7, 5, 5, 9, 10, 9, 10, 9, 5, 2, 7, 12, 5, 19, 23, 8, 10, 9, 1, 22, 4, 5, 18, 19, 8, 13, 26, 5, 10, 10, 12, 20, 7, 18, 5, 14, 3, 6, 4, 16, 19, 15, 1, 14, 10, 14, 23, 15, 11, 9, 2, 6, 6, 3, 10, 6, 21, 8, 2, 13, 2, 6, 2, 1, 9, 3, 4, 8, 9, 1, 15, 21, 3, 10, 11, 9, 4, 17, 12, 16, 13, 9, 19, 2, 10, 9, 16, 16, 16, 21, 21, 17, 13, 16, 15, 8, 17, 12, 16, 14, 2, 13], [36, 0, 

In [115]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

save_obj(batsmen_record, 'batsmen_record')
save_obj(bowlers_record, 'bowlers_record')

In [318]:
# stad_names = list(stad_avg.keys())
# keys = pd.Series(stad_names, dtype="category")
# vals = [float(len(stad_avg[k])) for k in keys.cat.categories]


# plt.figure(figsize=(20, 10))
# sns.barplot(x=keys.cat.codes, y=vals)

In [319]:
# dict(enumerate(keys.cat.categories))

In [100]:
def get_venue_score(df):
    venue = df.iloc[1]['venue']
    #assign venue scores here

    return stad_rating[venue]

def get_batsmen_scores(u_bat):
    rating = []

    for batsman in u_bat:
        try:
            rating.append(np.mean(batsmen_record[batsman][0]))
        except Exception as e:
            rating.append(42)
        
    return rating
        
def get_bowlers_scores(u_bowlers):
    rating = []

    for bowler in u_bowlers:
        try:
            rating.append(np.mean(bowlers_record[bowler][0]))
        except Exception as e:
            rating.append(30)
        
    return rating

def get_wickets(df):
    return len(get_batsmen_scores(df)) - 2

In [101]:
#For cross validation, use the last of the req_df as test data and use the rest for training
test = req_df[len(req_df) - 1]
train_set = req_df

#test_x is the test input similar to actual input
test_batsmen = list(set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['striker'].unique()) | set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['non_striker'].unique()))
test_bowlers = test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['bowler'].unique()
test_x = pd.DataFrame({'venue': [test.iloc[2]['venue']], 'innings': [1], 'batting_team': [test.iloc[2]['batting_team']], 'bowling_team':[test.iloc[2]['bowling_team']], 'batsmen': [test_batsmen], 'bowlers': [test_bowlers]})
test_y = [df.loc[(df['ball'] < 6.1) & (df['innings'] == 1)]['total_runs'].sum()]

#used to find the innings of the training match we are going to use for training
team_a = test.iloc[1]['batting_team']
team_b = test.iloc[1]['bowling_team']

train_x = []
train_y = []

for df in train_set:
    match_id = df.index[0]

    #innings for which we are going to use for training
    inn = df[(df['batting_team'] == team_a) & (df['bowling_team'] == team_b)]['innings'].iloc[0] 

    runs_x = df.loc[(df['ball'] < 6.1) & (df['innings'] == inn)]['total_runs'].sum() #runs of training innings
    runs_target = df.loc[(df['ball'] < 6.1) & (df['innings'] == 3 - inn)]['total_runs'].sum() #runs of other innings
    
    #MODEL CODE GOES HERE


    #convert the df into usable training data with required features
    df_train_inn = df.loc[(df['ball'] < 6.1) & (df['innings'] == inn)]

    # to add bowler rating: 'bowlers_rating': [numpy.array(get_bowlers_scores(df_train_inn))] 
    temp = pd.DataFrame({'venue': [get_venue_score(df)], 'batsmen_rating': [np.sum(numpy.array(get_batsmen_scores(df_train_inn)))], 'wickets': [get_wickets(df_train_inn)], 'other_runs': [runs_target], 'chasing': [0 if inn == 1 else 1]})

    train_x.append(temp.values.tolist()[0])
    train_y.append(runs_x)

print(train_x)
print(train_y)

IndexError: list index out of range

In [62]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [63]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [64]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_x)
    rmse= np.sqrt(-cross_val_score(model, train_x, train_y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [582]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 13.3384 (5.5679)



In [594]:
from sklearn.linear_model import Lasso

#Initializing the Lasso Regressor with Normalization Factor as True
lasso_reg = Lasso(normalize=True)

#Fitting the Training data to the Lasso regressor

lasso_reg.fit(train_x[1:], train_y[1:])

#Predicting for X_test
y_pred_lass =lasso_reg.predict([train_x[6]])

#Printing the Score with RMLSE
print(y_pred_lass, train_y[6])

[46.4733186] 56


NameError: name 'L_layer_model' is not defined